# Lineaire Regressie in Python
## 1.4 Gradient Descent

De Normal Equation heeft enkele beperkingen in gebruik, met name voor heel grote datasets:

- voor deze analytische afleiding van de regressiecoefficienten moeten we de inverse nemen over $m \times m$-matrix waarbij $m$ het aantal features/kolommen is in de input matrix $X$. Het bepalen van een inverse is een functie van de orde $O(n^3)$, dus bij grote waarden van $m$ wordt het te kostbaar om deze aanpak te gebruiken. 
- als datasets niet in het intern geheugen passen door een te groot aantal rijen wordt het inefficient.
- de Normal Equation werkt voor zover bekend alleen voor de standaard least squares kostenfunctie. We gaan binnenkort kijken naar andere kostenfuncties zoals logistische kostenfuncties waarvoor de Normal Equation niet werkt.

Een andere populaire aanpak om een kostenfunctie te optimaliseren is Gradient Descent. Volgens deze methode worden de coefficienten iteratief met kleine stapjes bijgesteld in de richting van een lagere waarde voor de kostenfunctie. Om de richting van de coefficienten te bepalen worden partial derivatives van de kostenfunctie gebruikt. Het voordeel is dat deze aanpak generiek bruikbaar is voor heel veel problemen en met grote hoeveelheden data. Een nadeel is dat Gradient Descent een lokaal minimum kan vinden in plaats van een globaal minimum. Daar komen we in een latere week op terug.

#### Gradient Desscent update functies

Het basis voor Gradient Descent is om de coefficienten stapsgewijs in de richting van het minimum te verplaatsen. We gebruiken daarvoor een algemene update rule, waarin $\theta_j$ de coefficient is die wordt geupdate, $J(\theta)$ de kostenfunctie die geparameteriseerd is door $\theta$ en $\alpha$ de learning rate die bepaalt hoe groot de stappen zijn waarmee geleerd wordt:

$$ \theta_j = \theta_j - \alpha \frac{\delta}{\delta \theta_j}J(\theta) $$

In deze update rule kunnen we het minteken verklaren door te kijken naar de afgeleide. Als de afgeleide positief is, dan ligt het minimum bij een lagere waarde van de coefficient en omgekeerd. De learning rate $\alpha$ bepaalt de grootte van de update stap. We gaan de learning rate later in detail bestuderen, voor nu geven we bij deze code een passende waarde voor de learning rate. 

We gaan eerst exerimenteren met de kostenfunctie $J(\theta)$ die we eerder hebben gebruikt om een lineaire regressielijn te fitten met een enkele onafhankelijke variable $x$. De aanpassing van de kostenfunctie met $\frac{1}{2n}$ maakt voor het optimalisatieprobleem niet uit, maar levert na het differentieren een nettere updaterule op.

$$ J(\theta) = \frac{1}{2n} \sum_{i = 1}^n (\widehat{y^{(i)}} - y^{(i)})^2 = \frac{1}{2n} \sum_{i = 1}^n (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)})^2 $$  

Door het toepassen van de chain-rule voor partitieel afgeleiden krijgen we dan:

$$ \frac{\delta J(\theta)}{\delta \theta_0} = \frac{1}{n} \sum_{i=1}^n (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot 1 $$

$$ \frac{\delta J(\theta)}{\delta \theta_1} = \frac{1}{n} \sum_{i=1}^n (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot x^{(i)} $$

De update rules worden dan:

$$ \theta_0 := \theta_0 - \alpha \cdot \frac{1}{n} \sum_{i=1}^n (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) $$
$$ \theta_1 := \theta_1 - \alpha \cdot \frac{1}{n} \sum_{i=1}^n (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot x^{(i)} $$
 

#### Implementatie van Gradient Descent

We laden wederom eerst de data en prepareren een matrix $X$ met input data en $y$ met de te verklaren variabele.

In [1]:
import pandas as pd
wijnen = pd.read_csv('winequality-red.csv', delimiter=';')

wijnen['bias'] = 1
X = wijnen[['bias', 'alcohol']].as_matrix()
y = wijnen.quality.as_matrix()

Daarnaast hebben we een vector $\theta$ nodig met coefficienten. In dit geval kunnen we alle coefficienten initialiseren op 0.

In [2]:
import numpy as np
theta = np.zeros((2))

We zullen eerst een eenvoudige uitwerking laten zien waarmee we de kostenfunctie berekenen over de trainingsset. De eenvoudige uitwerking gebruikt een `for`-loop om de kosten te verhogen met de kwadratische afwijking van een trainingspaar.

In [3]:
def computeUpdate(X, y):
    update0 = 0
    update1 = 0
    for xi, yi in zip(X, y):
        y_hat = xi[1] * theta[1] + theta[0]
        diff = (y_hat - yi)
        update0 -= diff
        update1 -= diff * xi[1]
    return update0 / len(X), update1 / len(X)

In [4]:
computeUpdate(X, y)

(5.6360225140712945, 59.153700229309955)

vervolgens kunnen we iteratief de update over de hele set berekenen en de coefficienten in $\theta$ daarmee updaten.

In [5]:
%%time
alpha = 0.01
theta = np.zeros((2))
for iter in range(50000):
    update0, update1 = computeUpdate(X, y)
    theta[0] += alpha * update0
    theta[1] += alpha * update1
print(theta)

[ 1.86411136  0.36187335]
CPU times: user 1min 4s, sys: 2.34 ms, total: 1min 4s
Wall time: 1min 4s


We zien direct 2 problemen met Gradient Descent. We krijgen een benadering van de meest optimale coefficienten en het model convergeert heel langzaam. We komen later terug op mogelijke verbeteringen. Voor de duidelijkheid hebben we voor nu op eenvoudig leesbare code gefocust, we kunnen de code een stuk efficienter maken als we deze herschrijven als matrix vermenervuldigingen met Numpy arrays.

In [6]:
def computeUpdate2(X, y):
    y_hat = np.dot(X, theta)
    costs = y_hat - y
    update0 = -np.sum(costs) / len(costs)
    update1 = -np.sum(costs * X[:, 1]) / len(costs)
    return update0, update1

We zien dat het qua verwerkingstijd heel veel uitmaakt hoe Gradient Descent wordt geimplementeerd. Met matrix producten gaat het ca. 80x sneller. Op een soortgelijke manier zijn er vele andere optimalisaties voor een snellere verwerking mogelijk, maar het principe van gradient descent blijft altijd hetzelfde.

In [7]:
%%time
alpha = 0.01
theta = np.zeros((2))
for iter in range(50000):
    update0, update1 = computeUpdate2(X, y)
    theta[0] += alpha * update0
    theta[1] += alpha * update1
print(theta)

[ 1.86411136  0.36187335]
CPU times: user 721 ms, sys: 0 ns, total: 721 ms
Wall time: 721 ms
